## Imports
- Place %%pycodestyle at the top of any cell to check python syntax

In [19]:
%load_ext pycodestyle_magic
%load_ext autoreload
%autoreload 2

# Common imports
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import shap

# Import from my GitHub
from getxgboostmodel.getxgboostmodel import get_xgboost_model
from randomizedgridsearch.randomizedgridsearch import RandomizedGridSearch
from transformers.transformers import *


The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize

In [22]:
# Example dataset
boston_data = load_boston()

# Extract pandas dataframe and target
X = pd.DataFrame(boston_data['data']).copy().values
y = pd.DataFrame(boston_data['target']).copy().values

# Train/test split
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.20, random_state=42)
train_X, test_X = train_X, test_X
train_y, test_y = train_y.reshape(-1, 1), test_y.reshape(-1, 1)

# An okay model fit to the data
try:
    xgb_model
except:
    xgb_model = get_xgboost_model(train_X, train_y)

# Pipeline
pipe = Pipeline([('standard_scaler', StandardScalerTransform()),
                 ('min_max_scaler', MinMaxScalerTransform()),
                 ('binarizer', BinarizerTransform()), 
#                  ('k_bins_discretizer', KBinsDiscretizerTransform()),
#                  ('k_bins_discretizer2', KBinsDiscretizerTransform2()),
#                  ('k_bins_discretizer3', KBinsDiscretizerTransform3()),                 
                 ('model', xgb_model)])

# Find the number of features
num_features = train_X.shape[1]

# Testing with these indices
indices = list(range(num_features))

# Scalers
scaler_values = [0.05]*num_features

# Possible configurations [None, True, or False] - None means not decided yet
param_distributions = {
    'standard_scaler': scaler_values,
    'min_max_scaler': scaler_values,
    'binarizer': scaler_values,    
}

experiments_results = pd.DataFrame()

## Run the Experiments

In [24]:
experiments_results

,standard_scaler__0,standard_scaler__1,standard_scaler__2,standard_scaler__3,standard_scaler__4,standard_scaler__5,standard_scaler__6,standard_scaler__7,standard_scaler__8,standard_scaler__9,...,binarizer__4,binarizer__5,binarizer__6,binarizer__7,binarizer__8,binarizer__9,binarizer__10,binarizer__11,binarizer__12,score
0,True,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,16.599612
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,15.989696
2,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,15.252662
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,14.035666
4,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,14.035666
5,False,False,False,False,True,False,False,True,False,False,...,True,False,True,False,False,False,False,False,False,9.074127
6,True,False,False,False,False,False,True,True,False,False,...,True,False,False,False,False,False,False,False,False,8.274537
7,True,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,8.274537
8,False,False,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,8.193766
9,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,7.828124


In [23]:
for iteration in range(20):
    # Randomly search the space n_iter times
    experiments_results_temp = RandomizedGridSearch(
        n_experiments=100,
        pipe=pipe,
        param_distributions=param_distributions,
        train_X=train_X,
        train_y=train_y,
        test_X=test_X,
        test_y=test_y,
        scoring='neg_mean_squared_error')
    
    # Append to experiment results
    experiments_results = experiments_results.append(experiments_results_temp, ignore_index=True)
    
    # Ignore past data
    # experiments_results = experiments_results_temp

    # Drop score
    experiments_X_df = experiments_results.drop(['score'], axis=1)

    # Get column names
    X_column_names = experiments_X_df.columns

    # Convert to numpy
    experiments_X = experiments_X_df.values
    experiments_y = experiments_results[['score']].values

    # Create an XGBoost model tuned with the experiments data
    try:
        xgb_experiments_model
        # Tune hyperparameters every once in a while
        # if iteration % 7 == 6:
        #     xgb_experiments_model = get_xgboost_model(experiments_X, experiments_y)
    except:
        xgb_experiments_model = get_xgboost_model(experiments_X, experiments_y)

    # Fit the model
    xgb_experiments_model.fit(experiments_X_df, experiments_y)

    # Extract shap values
    explainer = shap.TreeExplainer(xgb_experiments_model)
    shap_values = explainer.shap_values(experiments_X_df)

    # Shap as dataframe
    shap_values_of_experiments = pd.DataFrame(shap_values, columns=X_column_names)
    shap_values_of_experiments['score'] = experiments_y

    # Function to support analysis
    def find_significance_from_experiments_results(importance_threshold=0.05, max_toggles_to_lock_per_series=5):
        temp_df = shap_values_of_experiments.drop(['score'], axis=1).copy()
        for i in range(0, len(shap_values_of_experiments.index)):
            for j in range(0, len(shap_values_of_experiments.columns)):
                if not experiments_results.iloc[i, j]:
                    temp_df.iloc[i, j] = -1 * shap_values_of_experiments.iloc[i, j]
        temp_df = temp_df.sum().sort_values()
        options_to_set_to_false = temp_df[temp_df > 0]
        options_to_set_to_true = temp_df[temp_df < 0]    
        sum_value = (options_to_set_to_false.sum() + abs(options_to_set_to_true.sum()))
        options_to_set_to_false = options_to_set_to_false / sum_value
        options_to_set_to_true = abs(options_to_set_to_true) / sum_value
        options_to_set_to_false = options_to_set_to_false[options_to_set_to_false > importance_threshold].sort_values(ascending=False)
        options_to_set_to_true = options_to_set_to_true[options_to_set_to_true > importance_threshold]
        return options_to_set_to_false[0:max_toggles_to_lock_per_series], options_to_set_to_true[0:max_toggles_to_lock_per_series]

    # Call function
    options_to_set_to_false, options_to_set_to_true = find_significance_from_experiments_results()

    # Make the set to true DF
    options_to_set_to_true_df = pd.DataFrame()
    transformation, value = None, None
    try:
        transformation_and_value = options_to_set_to_true.keys()
    except:
        transformation, value = [], []
    if len(transformation_and_value) > 0:
        options_to_set_to_true_df["category"] = [x.split("__")[0] for x in transformation_and_value]
        options_to_set_to_true_df["transformation"] = [x.split("__")[1] for x in transformation_and_value]
        options_to_set_to_true_df["value"] = [x.split("__")[2] for x in transformation_and_value]
    else:
        options_to_set_to_true_df["category"] = []
        options_to_set_to_true_df["transformation"] = []
        options_to_set_to_true_df["value"] = []
    options_to_set_to_true_df["significance"] = options_to_set_to_true.values

    # Make the false DF
    options_to_set_to_false_df = pd.DataFrame()
    transformation, value = None, None
    try:
        transformation_and_value = options_to_set_to_false.keys()
    except:
        transformation, value = [], []
    if len(transformation_and_value) > 0:
        options_to_set_to_false_df["category"] = [x.split("__")[0] for x in transformation_and_value]
        options_to_set_to_false_df["transformation"] = [x.split("__")[1] for x in transformation_and_value]
        options_to_set_to_false_df["value"] = [x.split("__")[2] for x in transformation_and_value]
    else:
        options_to_set_to_true_df["category"] = []
        options_to_set_to_false_df["transformation"] = []
        options_to_set_to_false_df["value"] = []
    options_to_set_to_false_df["significance"] = options_to_set_to_false.values

    # Reset updates
    updates = 0
    
    # Set to True
    for index, row in options_to_set_to_true_df.iterrows():
        if param_distributions[row['category']][int(row['value'])] == None:
            updates += 1
        param_distributions[row['category']][int(row['value'])] = row['transformation'] + "__true"

    # Set to False  TEMP COMMENT OUT
#     for index, row in options_to_set_to_false_df.iterrows():
#         if param_distributions[row['category']][int(row['value'])] == None:
#             updates += 1        
#         param_distributions[row['category']][int(row['value'])] = row['transformation'] + "__false"

    # End early if there is nothing to tune
    print("Updates:", updates)
    if updates == 0:
        break
    
print("Done")

100%|██████████| 100/100 [00:04<00:00, 22.38it/s]


IndexError: list index out of range

In [ ]:
options_to_set_to_false, options_to_set_to_true

In [ ]:
param_distributions

## Final Results

In [ ]:
experiments_results.sort_values(by='score')